In [1]:
%%latex
\tableofcontents

<IPython.core.display.Latex object>

# Ambulance_Dispatch_2024_Reduce_Dimensionality

## Goals

- The goal of this notebook is to reduce the dimensionality of each dataset by identifying and dropping features that are well predicted by the remaining features by multicollinearity.

- In the next notebook we will build models on three groups of features:
    - Easy:  Features that the emergency dispatcher already has or can, without really precise maps, determine from the location
    - Medium:  Additionally, features that the emergency dispatcher can determine from precise maps and information from the cell service provider about the primary user of the phone
    - Hard:  Additionally, features about the vehicle that can be learned only by correlating information about the identity of the likely driver with vehicle registration and/or insurance records.  Not easily available in real time without lots of preparation, raises privacy concerns, and not likely to be very accurate

- The outbook of this notebook is the three sets of features, expressed as a dummy variable for each binned value of the feature, after dimensionality reduction.



## Methods

- For each of [Easy, Medium, Hard]:
    - For each feature:
        - Create a linear model (LinearRegression from sklearn) mapping the other features onto this feature.
        - Find the $R^2$ score from fitting the model:  r2 = LinearRegression().fit(X, y).score(X, y) where y is this feature and X is all of the other features.  
        - If $R^2=1$, then this feature is perfectly predicted by the other features.  
        - If this $R^2$ score is high, like greater than 0.9, then this feature is well predicted by other features, and we should consider dropping it.
        - The reason to not drop all features with high $R^2$ scores in the same step is that two (or more) features could be highly collinear, and we may only want to drop one of them.  If we drop one of them and recalculate, we may see that the other(s) now have low $R^2$ scores.

    - While the $\max(R^2) > 0.90$, run the above method and drop the feature with the highest $R^2$ score.
        - This choice of threshold of 0.90 is somewhat arbitary, perhaps arbitrarily high, and testing the results of different choices is an opportunity for future research.

    - Transform the features, which have 2-10 values, into 1-9 dummy features.  We choose to drop the first value because each of the dummy features of an original feature would be perfectly collinear with the others.

    - Repeat the ``While the $\max(R^2) > 0.90$,...'' process to reduce the number of features.  
    
    - Write the reduced dummy-variable features to file to use in the next notebook, where we will build models predicting whether each crash person needs an ambulance.
    
- We considered using Principal Component Analysis, but decided on $R^2$ instead.


# Setup

## Import Libraries

In [2]:
print ('Install Packages')

import sys, copy, math, time, os

print ('Python version: {}'.format(sys.version))

import numpy as np
print ('NumPy version: {}'.format(np.__version__))
np.set_printoptions(suppress=True)

import pandas as pd
print ('Pandas version:  {}'.format(pd.__version__))
pd.set_option('display.max_rows', 500)

import sklearn
print ('sklearn version: {}'.format(sklearn.__version__))
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# Set Randomness.  
import random
#np.random.seed(42) # NumPy
#random.seed(42) # Python
#tf.random.set_seed(42) # Tensorflow

import warnings
warnings.filterwarnings('ignore')

print ('Finished Installing Packages')

Install Packages
Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:51:49) [Clang 16.0.6 ]
NumPy version: 1.26.4
Pandas version:  2.2.2
sklearn version: 1.5.0
Finished Installing Packages


## Get Data

This function pulls in the saved output from Ambulance_Dispatch_2024_03_Impute_Missing_Data.

In [3]:
def Get_Data(target):
    print ('Get_Data')
    
    data = pd.read_csv('../../Big_Files/CRSS_Imputed_Data.csv', index_col=None)
    
    print ('data.shape = ', data.shape)
    data = data.reindex(sorted(data.columns), axis=1)
    first_column = data.pop(target) 
    data.insert(0, target, first_column)     

    return data

#data =  Get_Data('HOSPITAL')

In [4]:
def Thin_to_Hard_Features(data):
    print ('Thin_to_Hard_Features()')

    Merge = [
        'CASENUM',
        'VEH_NO',
        'PER_NO',        
    ]

    Accident = [
        'DAY_WEEK',
        'HOUR',
        'INT_HWY',
        'LGT_COND',
        'MONTH',
#        'PEDS',
        'PERMVIT',
#        'PERNOTMVIT', # Pedestrians, which we have taken out
        'PJ',
        'PSU',
        'PVH_INVL',
        'REGION',
        'REL_ROAD',
        'RELJCT1',
        'RELJCT2',
#        'SCH_BUS',
        'TYP_INT',
        'URBANICITY',
        'VE_FORMS',
        'VE_TOTAL',
        'WEATHER',
        'WRK_ZONE',
#        'YEAR',
    ]
    
    Vehicle = [
        'BODY_TYP',
#        'BUS_USE',
#        'EMER_USE',
        'MAKE',
#        'MOD_YEAR',
        'MODEL',
        'NUMOCCS',
        'VALIGN',
#        'VNUM_LAN',
        'VPROFILE',
        'VSPD_LIM',
#        'VSURCOND',
        'VTRAFCON',
        'VTRAFWAY',
    ]
    
    Person = [
        'AGE',
#        'LOCATION', # Pedestrian location; taken out
        'PER_TYP',
        'SEX',
        'HOSPITAL',    
    ]

    Engineered = [
        'VEH_AGE',
    ]
    
    # Put features in alphabetical order
    Features = Accident + Vehicle + Person + Engineered
    Features = sorted(Features)

    print ('Removed Features')
    for feature in data:
        if feature not in Features:
            print (feature)
    print ()
    
    data = data.filter(Features, axis=1)
    
    print ('data.shape: ', data.shape)
    
    print ('End Thin_to_Hard_Features()')
    print ()
        
    return data

def Test_Thin_to_Hard_Features():
    data = Get_Data()
    data = Thin_to_Hard_Features(data)
    for feature in data:
        display(data[feature].value_counts())
        
#Test_Thin_to_Hard_Features()

In [5]:
def Thin_to_Medium_Features(data):
    print ('Thin_to_Medium_Features()')

    Accident = [
        'DAY_WEEK',
        'HOUR',
        'INT_HWY',
#        'LGT_COND',
        'MONTH',
#        'PEDS',
#        'PERMVIT',
#        'PERNOTMVIT',
        'PJ',
        'PSU',
#        'PVH_INVL',
        'REGION',
        'REL_ROAD',
        'RELJCT1',
#        'RELJCT2',
#        'SCH_BUS',
        'TYP_INT',
        'URBANICITY',
#        'VE_FORMS',
#        'VE_TOTAL',
        'WEATHER',
#        'WRK_ZONE',
#        'YEAR',
    ]
    
    Vehicle = [
#        'BODY_TYP',
#        'BUS_USE',
#        'EMER_USE',
#        'MAKE',
#        'MOD_YEAR',
#        'MODEL',
#        'NUMOCCS',
        'VALIGN',
#        'VNUM_LAN',
        'VPROFILE',
        'VSPD_LIM',
#        'VSURCOND',
        'VTRAFCON',
        'VTRAFWAY',
    ]
    
    Person = [
        'AGE',
#        'LOCATION',
#        'PER_TYP',
        'SEX',
        'HOSPITAL',    
    ]

    Engineered = [
#        'VEH_AGE',
    ]
    
    # Put features in alphabetical order
    Features = Accident + Vehicle + Person + Engineered
    Features = sorted(Features)
    
    print ('Removed Features')
    for feature in data:
        if feature not in Features:
            print (feature)
    print ()
    
    data = data.filter(Features, axis=1)
    
    print ('data.shape: ', data.shape)
    
    print ('End Thin_to_Medium_Features()')
    print ()
        
    return data

def Test_Thin_to_Medium_Features():
    data = Get_Data()
    data = Thin_to_Medium_Features(data)
    for feature in data:
        display(data[feature].value_counts())
        
#Test_Thin_to_Medium_Features()

In [6]:
def Thin_to_Easy_Features(data):
    print ('Thin_to_Easy_Features()')

    Accident = [
        'DAY_WEEK',
        'HOUR',
#        'INT_HWY',
#        'LGT_COND',
        'MONTH',
#        'PEDS',
#        'PERMVIT',
#        'PERNOTMVIT',
        'PJ',
        'PSU',
#        'PVH_INVL',
        'REGION',
#        'REL_ROAD',
#        'RELJCT1',
#        'RELJCT2',
#        'SCH_BUS',
#        'TYP_INT',
        'URBANICITY',
#        'VE_FORMS',
#        'VE_TOTAL',
        'WEATHER',
#        'WRK_ZONE',
#        'YEAR',
    ]
    
    Vehicle = [
#        'BODY_TYP',
#        'BUS_USE',
#        'EMER_USE',
#        'MAKE',
#        'MOD_YEAR',
#        'MODEL',
#        'NUMOCCS',
#        'VALIGN',
#        'VNUM_LAN',
#        'VPROFILE',
#        'VSPD_LIM',
#        'VSURCOND',
#        'VTRAFCON',
#        'VTRAFWAY',
    ]
    
    Person = [
#        'AGE',
#        'LOCATION',
#        'PER_TYP',
#        'SEX',
        'HOSPITAL',    
    ]

    Engineered = [
#        'VEH_AGE',
#        'AGE_x_SEX',
#        'AGE_x_SCH_BUS'
    ]
    
    # Put features in alphabetical order
    Features = Accident + Vehicle + Person + Engineered
    Features = sorted(Features)

    print ('Removed Features')
    for feature in data:
        if feature not in Features:
            print (feature)
    print ()
        
    data = data.filter(Features, axis=1)
    
    print ('data.shape: ', data.shape)
    
    print ('End Thin_to_Easy_Features()')
    print ()
        
    return data

def Test_Thin_to_Easy_Features():
    data = Get_Data()
    data = Thin_to_Easy_Features(data)
    for feature in data:
        display(data[feature].value_counts())
        
#Test_Thin_to_Easy_Features()

In [7]:
def Get_Dummies(data, target):
    print ('Get_Dummies')
    print (data.shape)
    data = data.astype('category')
    Target = data.pop(target)
    
    data_Dummies = pd.get_dummies(data, prefix = data.columns, drop_first = True)

    # Use this version if the dataset has "99" signifying "Missing/Unknown",
    # but not if missing values have already been imputed.
#    data_Dummies = pd.get_dummies(data, prefix = data.columns, drop_first = False)
#    for feature in data_Dummies:
#        if '99' in feature:
#            data_Dummies.drop(columns=[feature], inplace=True)

    data_Dummies = data_Dummies.join(Target)
#    for feature in data_Dummies:
#        print (feature)
    print (data_Dummies.shape)
    print ()
    

    return data_Dummies

#data = Get_Dummies(data, 'HOSPITAL')

In [8]:
def Principal_Component_Analysis(data):
    print ('Principal_Component_Analysis()')
    Features = [feature for feature in data]
    n_components=300
    pca = PCA(n_components)
    print ('pca.fit()')
    pca.fit(data)
    
    data_pca = pca.transform(data)
    print (data_pca.shape)
    print (data_pca[:10])
    print ()
    data_pca = np.ascontiguousarray(data_pca)
    print (data_pca.shape)
    print (data_pca[:10])
    print ()
    data_pca = pd.DataFrame(data_pca) #, columns=['PCA%i' % i for i in range(n_components)], index=data.index)
    print (data_pca.head())
    print (data_pca.shape)
    print ()
    

    return data_pca

In [9]:
# Adapted from https://towardsdatascience.com/statistics-in-python-collinearity-and-multicollinearity-4cc4dcd82b3f
def calculate_vif(df, features):    
    r2_Dict, tolerance, vif = {}, {}, {}
    # all the features that you want to examine
    for feature in features:
        # extract all the other features you will regress against
        X = [f for f in features if f != feature]        
        X, y = df[X], df[feature]
        # extract r-squared from the fit
        r2 = LinearRegression().fit(X, y).score(X, y)
        r2_Dict[feature] = r2
        # calculate tolerance
        tolerance[feature] = 1 - r2
        # calculate VIF
        if tolerance[feature] !=0:
            vif[feature] = 1/(tolerance[feature])
        else:
            vif[feature] = 10000

    return pd.DataFrame({'r2': r2_Dict, 'Tolerance': tolerance, 'VIF': vif}), tolerance, r2_Dict

In [10]:
# Iteratively remove the feature with the largest VIF ('Variance Inflaction Factor')
# until the largest VIF is 10, or smallest Tolerance is 0.1, or largest R^2 is 0.9
def Reduce_Dimensionality(data, target):
    Target = data.pop(target)
    Features = [feature for feature in data]
    VIF, Tolerance_Dict, r2_Dict = calculate_vif(data, Features)
    Max_r2_Feature = VIF['r2'].idxmax()
    display(VIF)
    print (Max_r2_Feature)
    if r2_Dict[Max_r2_Feature] > 0.9:
        data.drop(columns = [Max_r2_Feature], inplace=True)
        print ('Drop ', Max_r2_Feature)
    print ()
    while r2_Dict[Max_r2_Feature] > 0.9:
        Features = [feature for feature in data]
        VIF, Tolerance_Dict, r2_Dict = calculate_vif(data, Features)
        Max_r2_Feature = VIF['r2'].idxmax()
        display(VIF)
        print (Max_r2_Feature)
        if r2_Dict[Max_r2_Feature] > 0.9:
            data.drop(columns = [Max_r2_Feature], inplace=True)
            print ('Drop ', Max_r2_Feature)
        print ()

    data = data.join(Target)
        
    return data
        
    

In [11]:
%%time
def Main():
    target = 'HOSPITAL'

    for i in range (3):
        data = Get_Data(target)
        data = data.astype('int64')
        if i==2:
            print ('Thin_to_Hard_Features()')
            data = Thin_to_Hard_Features(data)
        if i==1:
            print ('Thin_to_Medium_Features')
            data = Thin_to_Medium_Features(data)
        if i==0:
            print ('Thin_to_Easy_Features')
            data = Thin_to_Easy_Features(data)
            
#    Features = [feature for feature in data]
#    VIF, VIF_Dict = calculate_vif(data, Features)
#    display(VIF)
#    print ()

        data = Reduce_Dimensionality(data, target)
    
        for feature in data:
            data[feature] = pd.to_numeric(data[feature])
        print (data.shape)

        data_dummies = Get_Dummies(data, target)
#        for feature in data_dummies:
#            print (feature)
        print (data.shape)
        print ()
        data_dummies = Reduce_Dimensionality(data_dummies, target)
#        for feature in data_dummies:
#            print (feature)
        print (data.shape)
        print ()
        
        
        if i==2:
            data.to_csv('../../Big_Files/CRSS_Reduced_Hard_Features.csv', index=False)
            print ()
            print ('CRSS_Reduced_Hard_Features.csv Written')
            print ()
        if i==1:
            data.to_csv('../../Big_Files/CRSS_Reduced_Medium_Features.csv', index=False)
            print ()
            print ('CRSS_Reduced_Medium_Features.csv Written')
            print ()
        if i==0:
            data.to_csv('../../Big_Files/CRSS_Reduced_Easy_Features.csv', index=False)
            print ()
            print ('CRSS_Reduced_Easy_Features.csv Written')
            print ()
    
    print ()
    print ('Finished!')
    
    
Main()

Get_Data
data.shape =  (817623, 67)
Thin_to_Easy_Features
Thin_to_Easy_Features()
Removed Features
ACC_TYPE
AGE
AIR_BAG
ALC_STATUS
BODY_TYP
CARGO_BT
DEFORMED
DR_ZIP
EJECTION
HARM_EV
HIT_RUN
IMPACT1
INJ_SEV
INT_HWY
J_KNIFE
LGT_COND
MAKE
MAK_MOD
MAN_COLL
MAX_SEV
MAX_VSEV
MODEL
M_HARM
NUMOCCS
NUM_INJ
NUM_INJV
PCRASH4
PCRASH5
PERMVIT
PER_TYP
PVH_INVL
P_CRASH1
P_CRASH2
RELJCT1
RELJCT2
REL_ROAD
REST_MIS
REST_USE
ROLINLOC
ROLLOVER
SEAT_POS
SEX
SPEC_USE
SPEEDREL
TOWED
TOW_VEH
TYP_INT
VALIGN
VEH_AGE
VE_FORMS
VE_TOTAL
VPROFILE
VSPD_LIM
VSURCOND
VTCONT_F
VTRAFCON
VTRAFWAY
WRK_ZONE

data.shape:  (817623, 9)
End Thin_to_Easy_Features()

(817623, 9)
Get_Dummies
(817623, 9)
(817623, 40)

(817623, 9)



,r2,Tolerance,VIF
DAY_WEEK_1,0.264405,0.735595,1.359444
DAY_WEEK_2,0.274139,0.725861,1.377674
DAY_WEEK_3,0.367973,0.632027,1.582211
DAY_WEEK_4,0.291846,0.708154,1.412122
HOUR_1,0.507268,0.492732,2.029500
HOUR_2,0.517712,0.482288,2.073449
HOUR_3,0.722258,0.277742,3.600467
HOUR_4,0.868739,0.131261,7.618424
HOUR_5,0.870694,0.129306,7.733611
HOUR_6,0.788340,0.211660,4.724565


PSU_3

(817623, 9)


CRSS_Reduced_Easy_Features.csv Written

Get_Data
data.shape =  (817623, 67)
Thin_to_Medium_Features
Thin_to_Medium_Features()
Removed Features
ACC_TYPE
AIR_BAG
ALC_STATUS
BODY_TYP
CARGO_BT
DEFORMED
DR_ZIP
EJECTION
HARM_EV
HIT_RUN
IMPACT1
INJ_SEV
J_KNIFE
LGT_COND
MAKE
MAK_MOD
MAN_COLL
MAX_SEV
MAX_VSEV
MODEL
M_HARM
NUMOCCS
NUM_INJ
NUM_INJV
PCRASH4
PCRASH5
PERMVIT
PER_TYP
PVH_INVL
P_CRASH1
P_CRASH2
RELJCT2
REST_MIS
REST_USE
ROLINLOC
ROLLOVER
SEAT_POS
SPEC_USE
SPEEDREL
TOWED
TOW_VEH
VEH_AGE
VE_FORMS
VE_TOTAL
VSURCOND
VTCONT_F
WRK_ZONE

data.shape:  (817623, 20)
End Thin_to_Medium_Features()

(817623, 20)
Get_Dummies
(817623, 20)
(817623, 80)

(817623, 20)



KeyboardInterrupt: 